In [5]:
from ipywidgets import FloatProgress


In [6]:
import torch
print("torch version:", torch.__version__)
print("cuda available:", torch.cuda.device_count())
print("cuda current device:", torch.cuda.current_device())


torch version: 1.8.1+cu101
cuda available: 4
cuda current device: 0


In [7]:
cuda_count = torch.cuda.device_count()
for i in range(cuda_count):
    print(torch.cuda.get_device_properties(i))
    print(torch.cuda.memory_reserved(i)/1000000, "MiB")
    print(torch.cuda.memory_allocated(i)/1000000, "MiB")
    print((torch.cuda.memory_reserved(i)-torch.cuda.memory_allocated(i))/1000000, "MiB")


_CudaDeviceProperties(name='GeForce GTX 1080 Ti', major=6, minor=1, total_memory=11178MB, multi_processor_count=28)
0.0 MiB
0.0 MiB
0.0 MiB
_CudaDeviceProperties(name='GeForce GTX 1080 Ti', major=6, minor=1, total_memory=11178MB, multi_processor_count=28)
0.0 MiB
0.0 MiB
0.0 MiB
_CudaDeviceProperties(name='GeForce GTX 1080 Ti', major=6, minor=1, total_memory=11178MB, multi_processor_count=28)
2357.198848 MiB
2344.371712 MiB
12.827136 MiB
_CudaDeviceProperties(name='GeForce GT 710', major=3, minor=5, total_memory=1993MB, multi_processor_count=1)
0.0 MiB
0.0 MiB
0.0 MiB


In [8]:
device = 2


In [9]:
src_text = [
    """ PG&E stated it scheduled the blackouts in response to forecasts for high winds amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."""
    # """(CNN)Wanted: film director, must be eager to shoot footage of golden lassos and invisible jets. CNN confirms that Michelle MacLaren is leaving the upcoming "Wonder Woman" movie (The Hollywood Reporter first broke the story). MacLaren was announced as director of the movie in November. CNN obtained a statement from Warner Bros. Pictures that says, "Given creative differences, Warner Bros. and Michelle MacLaren have decided not to move forward with plans to develop and direct \'Wonder Woman\' together." (CNN and Warner Bros. Pictures are both owned by Time Warner.) The movie, starring Gal Gadot in the title role of the Amazon princess, is still set for release on June 23, 2017. It\'s the first theatrical movie centering around the most popular female superhero. Gadot will appear beforehand in "Batman v. Superman: Dawn of Justice," due out March 25, 2016. In the meantime, Warner will need to find someone new for the director\'s chair."""
]


In [10]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

model_name = 'google/pegasus-xsum'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

batch = tokenizer(src_text, truncation=True, padding='longest', return_tensors="pt").to(device)

enc_out = model.model.encoder(**batch)

cls_vec = enc_out[0][0][-1]
token_len = len(enc_out[0][0])
idx_product_list = []
for i in range(token_len):
    idx_product_list.append((i, torch.dot(cls_vec, enc_out[0][0][i]).item()))
idx_product_list.sort(key = lambda x: x[1], reverse=True)
for j, v in idx_product_list:
    print("{:.4f}\t{}\t{}".format(v, j, tokenizer.convert_ids_to_tokens(batch['input_ids'][0][j].item())))

28.8107	54	</s>
27.0433	53	.
26.2848	37	▁be
19.3284	42	s
18.7942	2	E
15.0912	29	.
14.5458	7	▁blackout
11.8792	33	▁customers
10.0990	19	.
9.5212	38	▁affected
9.4782	24	▁reduce
7.9273	4	▁it
7.7998	15	▁winds
7.6458	0	▁PG
7.3690	28	▁wildfires
7.3298	17	▁dry
7.2430	1	&
6.9584	25	▁the
6.9111	40	▁the
6.7592	6	▁the
6.7382	41	▁shutoff
6.5729	10	▁response
6.4962	26	▁risk
6.2655	51	▁midday
6.2544	48	▁through
6.1635	32	▁thousand
5.9423	35	▁scheduled
5.9383	31	▁800
5.8047	18	▁conditions
5.7208	21	▁aim
5.6878	14	▁high
5.6273	12	▁forecasts
5.0395	47	▁last
5.0256	52	▁tomorrow
4.8952	44	▁were
4.7502	11	▁to
4.3471	8	s
4.2328	13	▁for
4.1643	22	▁is
4.1346	45	▁expected
3.5453	5	▁scheduled
3.5214	30	▁Nearly
3.1741	39	▁by
2.7442	34	▁were
2.1745	50	▁least
2.0587	43	▁which
1.7573	16	▁amid
1.6381	3	▁stated
1.2800	46	▁to
0.9427	20	▁The
0.8546	36	▁to
0.3840	23	▁to
-0.2963	27	▁of
-1.7442	9	▁in
-2.4669	49	▁at


In [18]:
mean_vec = enc_out[0][0].mean(axis=0)
token_len = len(enc_out[0][0])
idx_product_list = []
for i in range(token_len):
    idx_product_list.append((i, torch.dot(mean_vec, enc_out[0][0][i]).item()))
idx_product_list.sort(key = lambda x: x[1], reverse=True)
for j, v in idx_product_list:
    print("{:.4f}\t{}\t{}".format(v, j, tokenizer.convert_ids_to_tokens(batch['input_ids'][0][j].item())))

10.6616	38	▁affected
10.4938	29	.
10.3702	17	▁dry
10.3474	42	s
10.2204	35	▁scheduled
10.0975	19	.
10.0763	48	▁through
9.9871	51	▁midday
9.8039	7	▁blackout
9.7272	12	▁forecasts
9.6942	45	▁expected
9.5551	18	▁conditions
9.2620	37	▁be
9.1937	14	▁high
8.9845	22	▁is
8.9196	39	▁by
8.9139	47	▁last
8.7090	53	.
8.7077	15	▁winds
8.5899	10	▁response
8.4092	26	▁risk
8.3601	52	▁tomorrow
8.3283	28	▁wildfires
8.3161	31	▁800
8.2324	2	E
8.1381	8	s
7.9391	5	▁scheduled
7.9051	21	▁aim
7.8567	41	▁shutoff
7.7405	33	▁customers
7.6190	32	▁thousand
7.5393	24	▁reduce
7.5040	1	&
7.3920	40	▁the
7.0216	54	</s>
6.9294	25	▁the
6.8644	34	▁were
6.8570	6	▁the
6.6960	4	▁it
6.6090	0	▁PG
6.5003	44	▁were
6.4019	30	▁Nearly
5.6822	11	▁to
5.1502	13	▁for
5.0386	43	▁which
4.9974	3	▁stated
3.2730	50	▁least
3.0036	46	▁to
2.7130	16	▁amid
2.5538	36	▁to
2.0470	23	▁to
1.5411	20	▁The
0.7791	9	▁in
-0.1784	27	▁of
-3.4650	49	▁at


In [6]:
from transformers import BartTokenizer, BartModel

model_name = 'facebook/bart-base'
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartModel.from_pretrained(model_name).to(device)

batch = tokenizer(src_text, truncation=True, padding='longest', return_tensors="pt").to(device)

enc_out = model.encoder(**batch)

cls_vec = enc_out[0][0][0]
token_len = len(enc_out[0][0])
idx_product_list = []
for i in range(token_len):
    idx_product_list.append((i, torch.dot(cls_vec, enc_out[0][0][i]).item()))
idx_product_list.sort(key = lambda x: x[1], reverse=True)
for j, v in idx_product_list:
    print("{:.4f}\t{}".format(v, tokenizer.convert_ids_to_tokens(batch['input_ids'][0][j].item())))

2.3107	<s>
0.5816	.
0.4279	</s>
0.2029	.
0.0860	.
-0.5289	Ġthe
-0.8352	Ġthe
-1.2149	Ġcustomers
-1.4052	Ġwildfires
-1.6172	ĠPG
-1.7019	&
-1.7182	ĠThe
-1.7678	Ġtomorrow
-1.7788	Ġwhich
-1.8951	Ġstated
-1.9752	E
-2.0115	Ġshut
-2.0888	Ġblack
-2.1348	offs
-2.1780	outs
-2.1870	Ġwinds
-2.2220	Ġthe
-2.3484	Ġit
-2.3862	Ġamid
-2.4060	Ġwere
-2.4280	Ġthousand
-2.4905	Ġconditions
-2.4916	ĠNearly
-2.5201	Ġis
-2.5815	Ġdry
-2.5861	Ġscheduled
-2.6492	Ġaffected
-2.6599	Ġmidday
-2.6624	Ġaim
-2.6733	Ġof
-2.7209	Ġby
-2.7477	Ġto
-2.7577	Ġreduce
-2.7892	Ġrisk
-2.7982	Ġin
-2.8137	Ġwere
-2.8759	Ġresponse
-2.8783	Ġ800
-2.8838	Ġto
-2.8966	Ġforecasts
-2.9695	Ġthrough
-3.0323	Ġfor
-3.0811	Ġlast
-3.0820	Ġleast
-3.0849	Ġat
-3.1023	Ġscheduled
-3.1732	Ġhigh
-3.3068	Ġexpected
-3.4139	Ġto
-3.5065	Ġbe
-3.5536	Ġto


In [6]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

model_name = 'google/pegasus-large'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

batch = tokenizer(src_text, truncation=True, padding='longest', return_tensors="pt").to(device)

enc_out = model.model.encoder(**batch)

cls_vec = enc_out[0][0][-1]
token_len = len(enc_out[0][0])
idx_product_list = []
for i in range(token_len):
    idx_product_list.append((i, torch.dot(cls_vec, enc_out[0][0][i]).item()))
idx_product_list.sort(key = lambda x: x[1], reverse=True)
for j, v in idx_product_list:
    print("{:.4f}\t{}".format(v, tokenizer.convert_ids_to_tokens(batch['input_ids'][0][j].item())))

36.7412	</s>
36.3622	'
36.2978	▁to
36.1380	▁to
36.0997	▁to
36.0924	▁the
36.0143	▁the
35.4002	'
34.9737	▁to
9.1097	▁In
7.6345	▁25,
7.6220	.
7.2346	▁23,
6.5750	.
6.5250	dot
6.4321	▁The
6.4048	)
6.3544	."
6.2566	▁will
6.1618	▁that
6.0490	▁for
5.8865	▁with
5.8709	▁is
5.8643	▁superhero
5.8603	▁are
5.8460	▁Justice
5.8273	▁(
5.7123	▁will
5.6116	▁Bros
5.5596	▁have
5.5450	▁in
5.4919	▁in
5.4499	▁Dawn
5.4226	▁title
5.4072	▁v
5.3927	▁due
5.3857	▁and
5.3665	▁by
5.3244	▁role
5.3039	dot
5.2910	▁the
5.2752	▁Superman
5.1884	▁the
5.1255	▁out
5.1174	▁2016.
5.1040	▁the
5.0763	▁theatrical
5.0401	▁June
5.0334	ren
5.0209	▁2017.
4.9435	▁centering
4.9184	La
4.9105	).
4.8763	▁the
4.8577	▁Ga
4.8201	▁princess
4.7930	'
4.7732	▁story
4.7570	▁need
4.7336	▁Pictures
4.7048	:
4.6988	▁It
4.6956	▁of
4.6586	Batman
4.6561	▁Mac
4.6430	▁meantime
4.6003	▁Bros
4.5873	▁Gal
4.5719	▁still
4.5643	,
4.5567	▁March
4.5389	▁forward
4.4981	,
4.4974	▁Warner
4.4894	▁release
4.4610	▁from
4.4480	▁set
4.4469	▁Bros
4.4324	▁a
4.3841	▁both
4.3

In [6]:
import datasets
datasets_list = datasets.list_datasets()
print(datasets_list)

['acronym_identification', 'ade_corpus_v2', 'adversarial_qa', 'aeslc', 'afrikaans_ner_corpus', 'ag_news', 'ai2_arc', 'air_dialogue', 'ajgt_twitter_ar', 'allegro_reviews', 'allocine', 'alt', 'amazon_polarity', 'amazon_reviews_multi', 'amazon_us_reviews', 'ambig_qa', 'ami', 'amttl', 'anli', 'app_reviews', 'aqua_rat', 'aquamuse', 'ar_cov19', 'ar_res_reviews', 'ar_sarcasm', 'arabic_billion_words', 'arabic_pos_dialect', 'arabic_speech_corpus', 'arcd', 'arsentd_lev', 'art', 'arxiv_dataset', 'ascent_kb', 'aslg_pc12', 'asnq', 'asset', 'assin', 'assin2', 'atomic', 'autshumato', 'babi_qa', 'banking77', 'bbaw_egyptian', 'bbc_hindi_nli', 'bc2gm_corpus', 'beans', 'best2009', 'bianet', 'bible_para', 'big_patent', 'billsum', 'bing_coronavirus_query_set', 'biomrc', 'biosses', 'blended_skill_talk', 'blimp', 'blog_authorship_corpus', 'bn_hate_speech', 'bookcorpus', 'bookcorpusopen', 'boolq', 'bprec', 'break_data', 'brwac', 'bsd_ja_en', 'bswac', 'c3', 'c4', 'cail2018', 'caner', 'capes', 'casino', 'catalo

In [8]:
dataset = datasets.load_dataset('cnn_dailymail', '3.0.0')
print(dataset)

  0%|          | 0/5 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/159M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/376M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/572k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/12.3M [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /home/naraki/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})


In [15]:
dataset['test'][6]

{'id': '027936fedb5e785fe79e84cb6e55c9cc26042ad3',
 'article': '(CNN)Wanted: film director, must be eager to shoot footage of golden lassos and invisible jets. CNN confirms that Michelle MacLaren is leaving the upcoming "Wonder Woman" movie (The Hollywood Reporter first broke the story). MacLaren was announced as director of the movie in November. CNN obtained a statement from Warner Bros. Pictures that says, "Given creative differences, Warner Bros. and Michelle MacLaren have decided not to move forward with plans to develop and direct \'Wonder Woman\' together." (CNN and Warner Bros. Pictures are both owned by Time Warner.) The movie, starring Gal Gadot in the title role of the Amazon princess, is still set for release on June 23, 2017. It\'s the first theatrical movie centering around the most popular female superhero. Gadot will appear beforehand in "Batman v. Superman: Dawn of Justice," due out March 25, 2016. In the meantime, Warner will need to find someone new for the director\

In [7]:
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
tgt_text[0]

'and Michelle MacLaren have decided not to move forward with plans to develop and direct \'Wonder Woman\' together." (CNN and Warner Bros.'